# 7. Export Videos of Segmentation

In [2]:
from napari_czifile2 import napari_get_reader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os 
import sys 
from tqdm import tqdm
import imageio
import cv2  # for resizing the frames
import warnings
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
warnings.filterwarnings("ignore")

In [3]:
plt.style.use('dark_background')

In [4]:
def random_colormap_transparent(mask):
    unique_labels = np.unique(mask)
    colored_mask = np.zeros((*mask.shape, 4), dtype=np.uint8)  # RGBA image

    # Assign random colors
    np.random.seed(0)  # Ensure reproducibility
    colors = np.random.randint(0, 255, size=(len(unique_labels), 3), dtype=np.uint8)
    colors = np.hstack((colors, np.full((len(unique_labels), 1), 255, dtype=np.uint8)))  # Add full opacity

    colors[0] = [0, 0, 0, 0]  # Make background transparent

    for i, label in enumerate(unique_labels):
        colored_mask[mask == label] = colors[i]

    return colored_mask

In [5]:
def set_min_max(channel, min_val, max_val):
    """
    Adjusts the contrast of the channel by setting the minimum and maximum values.
    Maps values in the range [min_val, max_val] to [0, 255].

    Args:
    - channel: Input image channel (2D array)
    - min_val: The minimum value to scale to
    - max_val: The maximum value to scale to

    Returns:
    - A contrast-adjusted channel, scaled to the range [0, 255]
    """
    # Ensure min_val and max_val are not the same, otherwise handle as special case
    if min_val == max_val:
        return np.zeros_like(channel)  # If min == max, return all zeros

    # Clip the values of the channel to the range [min_val, max_val]
    clipped = np.clip(channel, min_val, max_val)

    # Rescale the clipped values to the range [0, 255]
    # Normalize to [0, 1] and then scale to [0, 255]
    rescaled = (clipped - min_val) / (max_val - min_val) * 255
    
    return np.clip(rescaled, 0, 255)  # Ensure the values remain in the [0, 255] range



## Export just cell segmentation 

In [6]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')

In [8]:
input = 'T79'
input = [d for d in raw_data_dirs if input in d][0]
print(f'Using {input} as input directory')
input_dir = os.path.join('../../../RNA-FISH-raw-data/', input)
assert os.path.exists(input_dir), 'Input directory does not exist'
czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
print(f"Found {len(czi_files)} czi files in {input_dir}")
print(czi_files)

Using 20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI as input directory
Found 5 czi files in ../../../RNA-FISH-raw-data/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI
['20250328 1 T79 sample 1.czi', '20250328 1 T79 sample 5.czi', '20250328 1 T79 sample 2.czi', '20250328 1 T79 sample 3.czi', '20250328 1 T79 sample 4.czi']


In [9]:
file_number = 0
image = czi_files[file_number]
image_path = os.path.join(input_dir, image)
reader = napari_get_reader(image_path)
if reader is not None:
    layer_data = reader(image_path)
    image_data, metadata, layer_type = layer_data[0]
    image_data = np.squeeze(image_data)
    print(f"Loaded {image_path}")
    print("Metadata:", metadata)
    print("Image shape:", image_data.shape)  

# Load ROIs 
all_rois_path = f'../results/{input}/{image.replace(".czi", "_rois.npy")}'
all_rois = np.load(all_rois_path)
print(f"Loaded {all_rois_path}")

Loaded ../../../RNA-FISH-raw-data/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1.czi
Metadata: {'rgb': False, 'channel_axis': 2, 'translate': (0.0, 0.0, 29259.974395751953, 31219.198837280273), 'scale': (1.0, 1.0, 0.0974884033203125, 0.0974884033203125), 'contrast_limits': None, 'name': ['AF546-T1', 'DAPI-T2', 'AF647-T2']}
Image shape: (79, 3, 2048, 2048)
Loaded ../results/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1_rois.npy


In [7]:
# Determine the DAPI channel index from metadata.
dapi_channel = [name for name in metadata['name'] if 'DAPI' in name][0]
dapi_index = metadata['name'].index(dapi_channel)
image_data_dapi = image_data[:, dapi_index, :, :]  # Select the DAPI channel
print(f"Image shape after selecting DAPI channel: {image_data_dapi.shape}")

# Scale factor (pixel size in nm or um, adjust if needed)
pixel_x = metadata['scale'][2]
scale_bar_length = 50 / pixel_x  # 50um in pixels 

# Video parameters
fps = 10
num_z_slices = image_data_dapi.shape[0]
print(f"Number of Z slices: {num_z_slices}")

# Resolution downscale factor (set < 1 to decrease resolution)
scale_factor = 1  # e.g., 0.5 will reduce resolution by half

Image shape after selecting DAPI channel: (202, 2048, 2048)
Number of Z slices: 202


In [8]:
frames = []  # list to hold each frame (as an RGB array)

# --- Create video frames ---
for z in tqdm(range(num_z_slices), desc="Creating frames", total=num_z_slices):
    # Get the 2D image slice for the DAPI channel.
    img = image_data_dapi[z, :, :]
    roi_slice = all_rois[z, :, :]

    # Generate the colored overlay for the ROIs.
    roi_overlay = random_colormap_transparent(roi_slice)

    # Create a new figure; no need to display it.
    fig, ax = plt.subplots(figsize=(4, 4), dpi=300)
    ax.axis('off')

    # Plot the DAPI image (contrast-stretched).
    ax.imshow(set_min_max(img, 0, 255), cmap='gray')
    # Overlay the random-colored ROIs.
    ax.imshow(roi_overlay)

    # Add a white scale bar at the bottom right.
    height, width = img.shape
    x_start = width - 50 - scale_bar_length
    x_end = width - 50
    y = height - 50  # 50 pixels from the bottom edge
    ax.plot([x_start, x_end], [y, y], color='white', lw=2)

    # Render the figure to a numpy array.
    canvas = FigureCanvas(fig)
    canvas.draw()
    width_canvas, height_canvas = canvas.get_width_height()
    buf = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8).reshape(height_canvas, width_canvas, 4)
    frame = buf[..., :3]  # Extract RGB channels
    plt.close(fig)  # Close the figure to free memory

    # Downscale the frame if needed.
    if scale_factor < 1:
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        frame = cv2.resize(frame, (new_width, new_height))
    
    frames.append(frame)

Creating frames: 100%|██████████| 202/202 [10:56<00:00,  3.25s/it]


In [9]:
# Save frames as video 
output_dir = f'../videos/{input}/'
os.makedirs(output_dir, exist_ok=True)
output_video_path = f'../videos/{input}/{image.replace(".czi", "_rois.mov")}'
writer = imageio.get_writer(
    output_video_path,
    fps=10,
    format='ffmpeg',
    output_params=["-vcodec", "prores_ks"]
)
for frame in frames:
    writer.append_data(frame)
writer.close()
print(f"Video saved to {output_video_path}")

Multiple -codec/-c/-acodec/-vcodec/-scodec/-dcodec options specified for stream 0, only the last option '-codec:v prores_ks' will be used.
Incompatible pixel format 'yuv420p' for codec 'prores_ks', auto-selecting format 'yuv422p10le'


Video saved to ../videos/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/20250307 B1 Sample 5 Stack_rois.mov


## Export movie of both segmentation and raw images

In [ ]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')

In [ ]:
input = 'T79'
input = [d for d in raw_data_dirs if input in d][0]
print(f'Using {input} as input directory')
input_dir = os.path.join('../../../RNA-FISH-raw-data/', input)
assert os.path.exists(input_dir), 'Input directory does not exist'
czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
print(f"Found {len(czi_files)} czi files in {input_dir}")
print(czi_files)

Using 20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI as input directory
Found 5 czi files in ../../../RNA-FISH-raw-data/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI
['20250328 1 T79 sample 1.czi', '20250328 1 T79 sample 5.czi', '20250328 1 T79 sample 2.czi', '20250328 1 T79 sample 3.czi', '20250328 1 T79 sample 4.czi']


In [ ]:
file_number = 0
image = czi_files[file_number]
image_path = os.path.join(input_dir, image)
reader = napari_get_reader(image_path)
if reader is not None:
    layer_data = reader(image_path)
    image_data, metadata, layer_type = layer_data[0]
    image_data = np.squeeze(image_data)
    print(f"Loaded {image_path}")
    print("Metadata:", metadata)
    print("Image shape:", image_data.shape)  

# Load ROIs 
all_rois_path = f'../results/{input}/{image.replace(".czi", "_rois.npy")}'
all_rois = np.load(all_rois_path)
print(f"Loaded {all_rois_path}")

'20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI'

In [11]:
channels_indices = [(i, metadata['name'][i]) for i in range(len(metadata['name']))]
channel_order = [2, 0, 1, 3]
channels_indices = [channels_indices[i] for i in channel_order]
channels = [channel for _, channel in channels_indices]
channel_indices = [i for i, _ in channels_indices]
channel_names = ["DAPI", "9E108", "LOC105282603", "9E116"]
channel_maxs = [255, 255, 255, 255]
channel_colors = ["grey", "magenta", "yellow", "cyan"]

print(f"Channels: {channels}")
print(f"Channel names: {channel_names}")
print(f"Channel indices: {channel_indices}")
print(f"Channel max values: {channel_maxs}")
print(f"Channel colors: {channel_colors}")

IndexError: list index out of range

In [ ]:
frames = []  # list to hold each frame (as an RGB array)

# --- Create video frames ---
for z in tqdm(range(num_z_slices)[:], desc="Creating frames", total=num_z_slices):
    # Get the 2D image slice for the DAPI channel.
    img = image_data_dapi[z, :, :]
    roi_slice = all_rois[z, :, :]

    # Generate the colored overlay for the ROIs.
    roi_overlay = random_colormap_transparent(roi_slice)

    # Create a new figure; no need to display it.
    fig, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=300)
    
    ax = axs[0]
    ax.axis('off')

    # Plot the DAPI image (contrast-stretched).
    ax.imshow(set_min_max(img, 0, 255), cmap='gray')
    # Overlay the random-colored ROIs.
    ax.imshow(roi_overlay)

    # Add a white scale bar at the bottom right.
    height, width = img.shape
    x_start = width - 50 - scale_bar_length
    x_end = width - 50
    y = height - 50  # 50 pixels from the bottom edge
    ax.plot([x_start, x_end], [y, y], color='white', lw=2)

    # Plot the image in the second subplot 
    ax = axs[1]
    rgb_image = np.zeros((image_data.shape[2], image_data.shape[3], 3), dtype=float)  # (H, W, 3)

    for i, channel in enumerate(channels): 
        channel_index = channel_indices[i]
        channel_max = channel_maxs[i]
        channel_color = channel_colors[i]
        
        channel_image = image_data[z, channel_index, :, :]
        channel_adjusted = set_min_max(channel_image, 0, channel_max)
        if channel_color == "magenta": 
            rgb_image[..., 0] += channel_adjusted  
            rgb_image[..., 2] += channel_adjusted  
        elif channel_color == "cyan": 
            rgb_image[..., 1] += channel_adjusted
            rgb_image[..., 2] += channel_adjusted
        elif channel_color == "yellow": 
            rgb_image[..., 0] += channel_adjusted
            rgb_image[..., 1] += channel_adjusted
        elif channel_color == "grey":
            rgb_image[..., 0] += channel_adjusted
            rgb_image[..., 1] += channel_adjusted
            rgb_image[..., 2] += channel_adjusted
    
    # Normalize composite image to [0,1]
    rgb_image = np.clip(rgb_image/255, 0, 1)

    # Display 
    ax.imshow(rgb_image)
    ax.axis("off")

    # Add a white scale bar at the bottom right.
    ax.plot([x_start, x_end], [y, y], color='white', lw=2)

    # Add label for channels 
    for i, channel_name in enumerate(channel_names):
        ax.text(10, 100 + i * 80, channel_name, color=channel_colors[i], fontsize=6, weight='bold') 

    # Adjust layout
    plt.tight_layout()
    
    # Render the figure to a numpy array.
    canvas = FigureCanvas(fig)
    canvas.draw()
    width_canvas, height_canvas = canvas.get_width_height()
    buf = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8).reshape(height_canvas, width_canvas, 4)
    frame = buf[..., :3]  # Extract RGB channels
    plt.close(fig)  # Close the figure to free memory

    # Downscale the frame if needed.
    if scale_factor < 1:
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        frame = cv2.resize(frame, (new_width, new_height))
    
    frames.append(frame)

Creating frames: 100%|██████████| 85/85 [04:36<00:00,  3.25s/it]


In [49]:
# Save frames as video 
output_dir = f'../videos/{input}/'
os.makedirs(output_dir, exist_ok=True)
output_video_path = f'../videos/{input}/{image.replace(".czi", "_rois_and_image.mov")}'
writer = imageio.get_writer(
    output_video_path,
    fps=10,
    format='ffmpeg',
    output_params=["-vcodec", "prores_ks"]
)
for frame in frames:
    writer.append_data(frame)
writer.close()
print(f"Video saved to {output_video_path}")

2025-04-01 17:36:59,992 [WARNING] IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1800, 900) to (1808, 912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Multiple -codec/-c/-acodec/-vcodec/-scodec/-dcodec options specified for stream 0, only the last option '-codec:v prores_ks' will be used.
Incompatible pixel format 'yuv420p' for codec 'prores_ks', auto-selecting format 'yuv422p10le'


Video saved to ../videos/20250328 5 P14 LOC603-b3-488 9E108-b1-546 9E116-b2-647 DAPI/20250328 5 ZP sample 4_rois_and_image.mov


## Export just cytoplasm segmentation

In [ ]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')

In [ ]:
input = 'U34'
input = [d for d in raw_data_dirs if input in d][0]
print(f'Using {input} as input directory')
input_dir = os.path.join('../../../RNA-FISH-raw-data/', input)
assert os.path.exists(input_dir), 'Input directory does not exist'
czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
print(f"Found {len(czi_files)} czi files in {input_dir}")
print(czi_files)

Using 20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI as input directory
Found 5 czi files in ../../../RNA-FISH-raw-data/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI
['20250328 1 T79 sample 1.czi', '20250328 1 T79 sample 5.czi', '20250328 1 T79 sample 2.czi', '20250328 1 T79 sample 3.czi', '20250328 1 T79 sample 4.czi']


In [ ]:
file_number = 2
image = czi_files[file_number]
image_path = os.path.join(input_dir, image)
reader = napari_get_reader(image_path)
if reader is not None:
    layer_data = reader(image_path)
    image_data, metadata, layer_type = layer_data[0]
    image_data = np.squeeze(image_data)
    print(f"Loaded {image_path}")
    print("Metadata:", metadata)
    print("Image shape:", image_data.shape)  

# Load ROIs 
all_rois_path = f'../results/{input}/{image.replace(".czi", "_rois.npy")}'
all_rois = np.load(all_rois_path)
print(f"Loaded {all_rois_path}")

'20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI'

In [8]:
# Determine the DAPI channel index from metadata.
dapi_channel = [name for name in metadata['name'] if 'DAPI' in name][0]
dapi_index = metadata['name'].index(dapi_channel)
image_data_dapi = image_data[:, dapi_index, :, :]  # Select the DAPI channel
print(f"Image shape after selecting DAPI channel: {image_data_dapi.shape}")

# Scale factor (pixel size in nm or um, adjust if needed)
pixel_x = metadata['scale'][2]
scale_bar_length = 50 / pixel_x  # 50um in pixels 

# Video parameters
fps = 10
num_z_slices = image_data_dapi.shape[0]
print(f"Number of Z slices: {num_z_slices}")

# Resolution downscale factor (set < 1 to decrease resolution)
scale_factor = 1  # e.g., 0.5 will reduce resolution by half

Image shape after selecting DAPI channel: (202, 2048, 2048)
Number of Z slices: 202


In [9]:
frames = []  # list to hold each frame (as an RGB array)

# --- Create video frames ---
for z in tqdm(range(num_z_slices), desc="Creating frames", total=num_z_slices):
    # Get the 2D image slice for the DAPI channel.
    img = image_data_dapi[z, :, :]
    roi_slice = all_rois[z, :, :]

    # Generate the colored overlay for the ROIs.
    roi_overlay = random_colormap_transparent(roi_slice)

    # Create a new figure; no need to display it.
    fig, ax = plt.subplots(figsize=(4, 4), dpi=300)
    ax.axis('off')

    # Plot the DAPI image (contrast-stretched).
    ax.imshow(set_min_max(img, 0, 255), cmap='gray')
    # Overlay the random-colored ROIs.
    ax.imshow(roi_overlay)

    # Add a white scale bar at the bottom right.
    height, width = img.shape
    x_start = width - 50 - scale_bar_length
    x_end = width - 50
    y = height - 50  # 50 pixels from the bottom edge
    ax.plot([x_start, x_end], [y, y], color='white', lw=2)

    # Render the figure to a numpy array.
    canvas = FigureCanvas(fig)
    canvas.draw()
    width_canvas, height_canvas = canvas.get_width_height()
    buf = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8).reshape(height_canvas, width_canvas, 4)
    frame = buf[..., :3]  # Extract RGB channels
    plt.close(fig)  # Close the figure to free memory

    # Downscale the frame if needed.
    if scale_factor < 1:
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        frame = cv2.resize(frame, (new_width, new_height))
    
    frames.append(frame)

Creating frames: 100%|██████████| 202/202 [09:46<00:00,  2.90s/it]


In [10]:
# Save frames as video 
output_dir = f'../videos/{input}/'
os.makedirs(output_dir, exist_ok=True)
output_video_path = f'../videos/{input}/{image.replace(".czi", "_cell_borders.mov")}'
writer = imageio.get_writer(
    output_video_path,
    fps=10,
    format='ffmpeg',
    output_params=["-vcodec", "prores_ks"]
)
for frame in frames:
    writer.append_data(frame)
writer.close()
print(f"Video saved to {output_video_path}")

Multiple -codec/-c/-acodec/-vcodec/-scodec/-dcodec options specified for stream 0, only the last option '-codec:v prores_ks' will be used.
Incompatible pixel format 'yuv420p' for codec 'prores_ks', auto-selecting format 'yuv422p10le'


Video saved to ../videos/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/20250307 B1 Sample 1 Stack_cell_borders.mov
